In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## အဆင့် ၁ - ဖွဲ့စည်းထားသော အထွက်များအတွက် Pydantic မော်ဒယ်များ သတ်မှတ်ပါ

ဤမော်ဒယ်များသည် အေးဂျင့်များ ပြန်ပေးမည့် **schema** ကို သတ်မှတ်ပေးပါသည်။ Pydantic နှင့်အတူ `response_format` ကို အသုံးပြုခြင်းဖြင့် အောက်ပါအကျိုးကျေးဇူးများ ရရှိနိုင်ပါသည် -
- ✅ အမျိုးအစားလုံခြုံသော ဒေတာထုတ်ယူမှု
- ✅ အလိုအလျောက် အတည်ပြုခြင်း
- ✅ အခမဲ့စာသားဖြင့် ပြန်လည်ဖြေကြားမှုမှ အမှားများမရှိခြင်း
- ✅ အကွက်များအပေါ် အခြေခံ၍ လွယ်ကူသော အခြေအနေအရ လမ်းကြောင်းသတ်မှတ်ခြင်း


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## အဆင့် ၂ - ဟိုတယ်အခန်းကြိုတင်မှာယူရေးကိရိယာကို ဖန်တီးပါ

ဒီကိရိယာက **availability_agent** က အခန်းရနိုင်မရနိုင် စစ်ဆေးဖို့ ခေါ်သုံးမယ့်အရာဖြစ်ပါတယ်။ `@ai_function` အလှဆင်ကို အသုံးပြုပြီး:
- Python function ကို AI-callable tool အဖြစ် ပြောင်းလဲပေးနိုင်သည်
- LLM အတွက် JSON schema ကို အလိုအလျောက် ဖန်တီးပေးသည်
- Parameter အတိအကျ စစ်ဆေးပေးသည်
- Agent များက အလိုအလျောက် ခေါ်သုံးနိုင်စေသည်

ဒီနမူနာအတွက်:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → အခန်းရရှိနိုင် ✅
- **အခြားမြို့အားလုံး** → အခန်းမရရှိ ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## အဆင့် ၃ - လမ်းကြောင်းသတ်မှတ်ရန် အခြေအနေဖန်တီးမှုအလုပ်ဆောင်ပုံများကို သတ်မှတ်ပါ

ဤအလုပ်ဆောင်ပုံများသည် Agent ရဲ့ တုံ့ပြန်မှုကို စစ်ဆေးပြီး workflow အတွင်းမှာ ဘယ်လမ်းကြောင်းကို သွားမလဲဆိုတာ ဆုံးဖြတ်ပေးပါသည်။

**အဓိက ပုံစံ:**
1. Message ကို `AgentExecutorResponse` ဖြစ်မဖြစ် စစ်ဆေးပါ။
2. ဖွဲ့စည်းထားသော output (Pydantic model) ကို ဖော်ထုတ်ပါ။
3. Routing ကို ထိန်းချုပ်ရန် `True` သို့မဟုတ် `False` ကို ပြန်ပေးပါ။

Workflow သည် **edges** တွင် အခြေအနေများကို သုံးပြီး ဘယ် executor ကို နောက်တစ်ခုအနေနဲ့ ခေါ်ယူမလဲဆိုတာ ဆုံးဖြတ်ပါမည်။


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## အဆင့် ၄ - အထူးပြင်ဆင်ထားသော Display Executor တစ်ခု ဖန်တီးပါ

Executors ဆိုတာက transformation တွေ၊ ဒါမှမဟုတ် side effects တွေကို လုပ်ဆောင်တဲ့ workflow components တွေဖြစ်ပါတယ်။ `@executor` decorator ကို အသုံးပြုပြီး နောက်ဆုံးရလဒ်ကို ပြသပေးမယ့် အထူးပြင်ဆင်ထားတဲ့ executor တစ်ခုကို ဖန်တီးနိုင်ပါတယ်။

**အဓိက အကြောင်းအရာများ:**
- `@executor(id="...")` - function တစ်ခုကို workflow executor အဖြစ် မှတ်ပုံတင်ပေးသည်
- `WorkflowContext[Never, str]` - input/output အတွက် type hints
- `ctx.yield_output(...)` - workflow ရလဒ်နောက်ဆုံးကို ထုတ်ပေးသည်


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## အဆင့် ၅ - ပတ်ဝန်းကျင် အပြောင်းအလဲများကို Load လုပ်ပါ

LLM client ကို ပြင်ဆင်ပါ။ ဤဥပမာသည် အောက်ပါအရာများနှင့်အလုပ်လုပ်ပါသည်။
- **GitHub Models** (GitHub token ဖြင့် အခမဲ့အဆင့်)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## အဆင့် ၆ - AI အေးဂျင့်များကို ဖွဲ့စည်းထားသော အထွေထွေထုတ်လွှင့်မှုများနှင့် ဖန်တီးခြင်း

**အထူးပြု အေးဂျင့် ၃ ခု** ကို ဖန်တီးပြီး `AgentExecutor` ဖြင့် ထုပ်ပိုးထားသည်။

1. **availability_agent** - ကိရိယာကို အသုံးပြု၍ ဟိုတယ်ရရှိနိုင်မှုကို စစ်ဆေးသည်
2. **alternative_agent** - အခန်းမရှိသောအခါ အခြားမြို့များကို အကြံပြုသည်
3. **booking_agent** - အခန်းရရှိနိုင်သောအခါ အခန်းကြိုတင်မှာရန် အားပေးသည်

**အဓိက အင်္ဂါရပ်များ:**
- `tools=[hotel_booking]` - အေးဂျင့်အတွက် ကိရိယာကို ပေးသည်
- `response_format=PydanticModel` - ဖွဲ့စည်းထားသော JSON output ကို အတိအကျထုတ်ရန် အတင်းအဓိပ်ပါယ်ထားသည်
- `AgentExecutor(..., id="...")` - အေးဂျင့်ကို workflow အသုံးပြုမှုအတွက် ထုပ်ပိုးထားသည်


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## အဆင့် ၇: အခြေအနေအရ ချိတ်ဆက်မှုများဖြင့် Workflow ကို တည်ဆောက်ပါ

အခုတော့ `WorkflowBuilder` ကို အသုံးပြု၍ အခြေအနေအရ လမ်းကြောင်းချိတ်ဆက်မှုများပါဝင်သော graph ကို တည်ဆောက်ပါမည်။

**Workflow ဖွဲ့စည်းမှု:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**အဓိက နည်းလမ်းများ:**
- `.set_start_executor(...)` - စတင်ဝင်ရောက်မည့်အချက်ကို သတ်မှတ်သည်
- `.add_edge(from, to, condition=...)` - အခြေအနေအရ ချိတ်ဆက်မှုကို ထည့်သွင်းသည်
- `.build()` - Workflow ကို နောက်ဆုံးအတည်ပြုသည်


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## အဆင့် ၈ - စမ်းသပ်မှုအမှတ် ၁ ကို အလုပ်လုပ်ပါ - Availability မရှိသော မြို့ (ပဲရစ်)

ပဲရစ်မြို့တွင် ဟိုတယ်များကို တောင်းဆိုခြင်းဖြင့် **availability မရှိခြင်း** လမ်းကြောင်းကို စမ်းသပ်ကြမယ် (ကျွန်ုပ်တို့ simulation မှာ အခန်းမရှိပါဘူး)။


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## အဆင့် ၉: စမ်းသပ်မှုအမှတ် ၂ - Availability ရှိသော မြို့ (Stockholm)

အခုတော့ **availability** လမ်းကြောင်းကို စမ်းသပ်ဖို့ Stockholm (ကျွန်ုပ်တို့ရဲ့ simulation မှာ အခန်းများရှိတဲ့မြို့) မှာ ဟိုတယ်များကို တောင်းဆိုပါ။


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## အဓိကအချက်များနှင့် နောက်တစ်ဆင့်များ

### ✅ သင်လေ့လာခဲ့တာများ:

1. **WorkflowBuilder Pattern**
   - `.set_start_executor()` ကို အသုံးပြု၍ စတင်မှုအချက်ကို သတ်မှတ်ပါ
   - `.add_edge(from, to, condition=...)` ကို အသုံးပြု၍ အခြေအနေအရ လမ်းကြောင်းသတ်မှတ်ပါ
   - `.build()` ကို ခေါ်၍ workflow ကို အပြီးသတ်ပါ

2. **Conditional Routing**
   - အခြေအနေ function များသည် `AgentExecutorResponse` ကို စစ်ဆေးသည်
   - ဖွဲ့စည်းထားသော output များကို ဖော်ထုတ်ပြီး လမ်းကြောင်းဆုံးဖြတ်ချက်များကို ချမှတ်ပါ
   - `True` ကို ပြန်ပေးပါက edge ကို အလုပ်လုပ်စေပြီး၊ `False` ကို ပြန်ပေးပါက ကျော်သွားသည်

3. **Tool Integration**
   - `@ai_function` ကို အသုံးပြု၍ Python function များကို AI tools အဖြစ် ပြောင်းပါ
   - Agents များသည် tool များကို လိုအပ်သောအခါ အလိုအလျောက် ခေါ်သည်
   - Tools များသည် agents ဖော်ထုတ်နိုင်သော JSON ကို ပြန်ပေးသည်

4. **Structured Outputs**
   - Pydantic models ကို အသုံးပြု၍ အချက်အလက်များကို အမျိုးအစားအတိုင်း လုံခြုံစွာ ဖော်ထုတ်ပါ
   - Agents ဖန်တီးရာတွင် `response_format=MyModel` ကို သတ်မှတ်ပါ
   - `Model.model_validate_json()` ဖြင့် response များကို ဖော်ထုတ်ပါ

5. **Custom Executors**
   - `@executor(id="...")` ကို အသုံးပြု၍ workflow components များ ဖန်တီးပါ
   - Executors များသည် အချက်အလက်များကို ပြောင်းလဲခြင်း သို့မဟုတ် side effects များကို ဆောင်ရွက်နိုင်သည်
   - `ctx.yield_output()` ကို အသုံးပြု၍ workflow ရလဒ်များကို ထုတ်ပေးပါ

### 🚀 အမှန်တကယ် အသုံးချနိုင်သော လျှောက်လွှာများ:

- **ခရီးသွားအကျိုးဆောင်မှု**: ရရှိနိုင်မှု စစ်ဆေးခြင်း၊ အခြားရွေးချယ်စရာများ အကြံပေးခြင်း၊ ရွေးချယ်မှုများ နှိုင်းယှဉ်ခြင်း
- **ဖောက်သည်ဝန်ဆောင်မှု**: ပြဿနာအမျိုးအစား၊ စိတ်ခံစားမှု၊ အရေးကြီးမှုအရ လမ်းကြောင်းသတ်မှတ်ခြင်း
- **E-commerce**: စတော့ရှယ်ယာ စစ်ဆေးခြင်း၊ အခြားရွေးချယ်စရာများ အကြံပေးခြင်း၊ အော်ဒါများ ဆောင်ရွက်ခြင်း
- **အကြောင်းအရာ စစ်ဆေးမှု**: Toxicity score များ၊ အသုံးပြုသူ flag များအရ လမ်းကြောင်းသတ်မှတ်ခြင်း
- **အတည်ပြုမှု Workflow များ**: ပမာဏ၊ အသုံးပြုသူအခန်းကဏ္ဍ၊ အန္တရာယ်အဆင့်အရ လမ်းကြောင်းသတ်မှတ်ခြင်း
- **အဆင့်များစွာ ပြုလုပ်မှု**: အချက်အလက်အရည်အသွေး၊ ပြည့်စုံမှုအရ လမ်းကြောင်းသတ်မှတ်ခြင်း

### 📚 နောက်တစ်ဆင့်များ:

- ပိုမိုရှုပ်ထွေးသော အခြေအနေများ (အချက်အလက်များစွာ) ထည့်သွင်းပါ
- Workflow အခြေအနေ စီမံခန့်ခွဲမှုဖြင့် loops များ ဆောင်ရွက်ပါ
- Reusable components များအတွက် sub-workflows များ ထည့်သွင်းပါ
- အမှန်တကယ် API များ (ဟိုတယ်ဘွတ်ကင်၊ စတော့ရှယ်ယာစနစ်များ) နှင့် ပေါင်းစည်းပါ
- Error handling နှင့် fallback လမ်းကြောင်းများ ထည့်သွင်းပါ
- Built-in visualization tools ဖြင့် workflows များကို ရှင်းလင်းစွာ ဖော်ပြပါ



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာရှိသော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအမှားများ သို့မဟုတ် အနားယူမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
